<a href="https://colab.research.google.com/github/mwl10/hetvae/blob/errors/src/catalina_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
os.chdir('/content')
! git clone --branch errors https://github.com/mwl10/hetvae
os.chdir('/content/hetvae')
! pip install -r requirements.txt
os.chdir('/content/hetvae/src')
import numpy as np
import torch
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import models
from argparse import Namespace
import torch.optim as optim
import utils
import my_utils
import pandas as pd
from glob import glob
import importlib
import vae_models
import matplotlib.pyplot as plt
import sys
from dataset import DataSet
%matplotlib inline

fatal: destination path 'hetvae' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
def define_model_args(trial):

    args = Namespace(
        batch_size = 8, #trial.suggest_categorical("batch_size", [8,16,32]),
        bound_variance = True,
        const_var = False,
        dataset='toy',
        dropout = 0.3,#trial.suggest_float("dropout", 0.0,0.5),
        elbo_weight = 1.0,#trial.suggest_float("elbo_weight", 0.0, 2.0),
        embed_time = trial.suggest_categorical("embed_time", [16,32,64]),
        enc_num_heads=trial.suggest_categorical("enc_num_heads", [4,8,16]),
        intensity=True,
        k_iwae=1,
        kl_annealing=False,#trial.suggest_categorical("kl_annealing",False),
        kl_zero=False, 
        latent_dim=trial.suggest_categorical("latent_dim", [64,128]),
        lr=trial.suggest_float("lr", 1e-7, 1e-1, log=True),
        mixing="concat_and_mix",#trial.suggest_categorical("mixing", ["concat", "concat_and_mix"]),#"separate", "interp_only", "na"]),
        mse_weight=4,#trial.suggest_float("mse_weight",1,20),
        #n=trial.suggest_categorical("n", [8,16,32,64,128]),
        net='hetvae', 
        niters=1000, 
        norm=True, 
        normalize_input='znorm', 
        num_ref_points=128,#trial.suggest_categorical("num_ref_points", [64, 128,256]),
        rec_hidden=32,#,trial.suggest_categorical("rec_hidden", [32,64, 128]),
        recon_loss=False, 
        sample_tp=0.5, #trial.suggest_float("sample_tp", 0.1,0.9), # will be ignored
        save=False, 
        seed=0, 
        shuffle=True, 
        std=0.1, 
        var_per_dim=False, 
        width=trial.suggest_categorical("width", [128])
    )

    return args

In [10]:
EPOCHS = 200
#FILES = glob('/content/hetvae/data/CAT/*/*')
#FILES = glob('/content/hetvae/data/CAT/*/*')[:50]

FILES = glob('/content/hetvae/data/CAT/*/*')[:2]
DIM = 1

In [11]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [13]:
def objective(trial):

    args = define_model_args(trial)

    seed = args.seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # dataset hyperparams
    num_samples=trial.suggest_int("num_samples", 1,6)
    # normalize choices for optuna
    #x_by_range = trial.suggest_categorical("x_by_range", [True, False])
    #y_by_range = False #trial.suggest_categorical("y_by_range", [True, False])
    #normalize_y = trial.suggest_categorical("normalize_y", ["all", "individual"])

    with suppress_stdout():
        # AGN = DataSet().add_files(FILES).files_to_numpy().handle_dups().prune_graham().resample_dataset(num_samples=num_samples) \
        #     .normalize(y_by_range=y_by_range, x_by_range=x_by_range).set_union_x().zero_fill().make_masks(frac=args.sample_tp)
        # AGN = DataSet()
        # AGN.dataset = lcs
        # AGN = AGN.handle_dups().prune_outliers().resample_dataset(num_samples=num_samples) \
        #      .normalize(y_by_range=y_by_range, x_by_range=x_by_range).set_union_x().zero_fill().make_masks(frac=args.sample_tp)
        Catalina = DataSet() \
                .add_files(FILES) \
                .files_to_numpy() \
                .handle_dups() \
                .prune_graham(plot=False, index=10, std_threshold=1) \
                .resample_dataset(num_samples=3) \
                .normalize(normalize_x = 'no', x_by_range=False) \
                .reorder() \
                .set_union_x() \
                .zero_fill() \
                .error_to_sample_weight()


    LightCurves = Catalina.dataset

    training, valid = np.split(LightCurves, [int(np.floor(1*len(LightCurves)))])# shuffle?
    
    
    train_loader = torch.utils.data.DataLoader(training, batch_size=args.batch_size)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=args.batch_size)
    
    
    net = models.load_network(args, DIM, torch.Tensor(Catalina.union_x)) # , device="cuda"
    

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam"])
    optimizer = getattr(optim, optimizer_name)(net.parameters(), lr=args.lr)
    frac = 0.4#trial.suggest_float("sample_tp", 0.1,0.9)
    step_size = trial.suggest_categorical("step_size", [200,300,400,500])
    gamma = trial.suggest_float("gamma",0,1)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=gamma, step_size=step_size)
    beta = trial.suggest_float("beta", 0,1)
    for epoch in range(EPOCHS):
        nll_loss, mse = my_utils.train(net, optimizer, epoch, train_loader, args, device=device, frac=frac, beta=beta)
        #nll_loss = my_utils.evaluate(net, valid_loader, device=device)
        trial.report(nll_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return nll_loss

In [14]:
study = optuna.create_study(direction="minimize")

[I 2022-06-23 08:35:27,478] A new study created in memory with name: no-name-12b6e9e5-fd77-43ca-b867-9f31295e7447


In [15]:
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]


Iter: 0, train loss: 14.2375, avg nll: 1.5266, avg kl: 14.0575, mse: 0.966364, mae: 0.794924
Iter: 100, train loss: 5.4709, avg nll: 1.4605, avg kl: 0.0314, mse: 0.997638, mae: 0.821115


[I 2022-06-23 08:35:42,869] Trial 0 finished with value: 1.2153575420379639 and parameters: {'embed_time': 64, 'enc_num_heads': 4, 'latent_dim': 128, 'lr': 0.00043085010961043, 'width': 128, 'num_samples': 4, 'optimizer': 'Adam', 'step_size': 500, 'gamma': 0.7886516703325803, 'beta': 0.6292328519616006}. Best is trial 0 with value: 1.2153575420379639.
/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]


Iter: 0, train loss: 11.4161, avg nll: 1.6318, avg kl: 12.5004, mse: 0.972452, mae: 0.797980
Iter: 100, train loss: 5.4557, avg nll: 1.4589, avg kl: 0.0068, mse: 0.998403, mae: 0.821155


[I 2022-06-23 08:35:57,945] Trial 1 finished with value: 1.4168927669525146 and parameters: {'embed_time': 16, 'enc_num_heads': 4, 'latent_dim': 128, 'lr': 0.0019072009227779911, 'width': 128, 'num_samples': 4, 'optimizer': 'Adam', 'step_size': 500, 'gamma': 0.6777106390711467, 'beta': 0.47154523116254143}. Best is trial 0 with value: 1.2153575420379639.
/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]


...

Iter: 0, train loss: 11.3380, avg nll: 1.6137, avg kl: 9.4275, mse: 0.980217, mae: 0.802023
Iter: 100, train loss: 7.4105, avg nll: 1.4678, avg kl: 3.1546, mse: 1.000208, mae: 0.819772


[I 2022-06-23 08:36:40,846] Trial 3 finished with value: 1.415003776550293 and parameters: {'embed_time': 32, 'enc_num_heads': 8, 'latent_dim': 128, 'lr': 4.187905926266953e-05, 'width': 128, 'num_samples': 2, 'optimizer': 'Adam', 'step_size': 500, 'gamma': 0.5674575242768608, 'beta': 0.6155825989267003}. Best is trial 0 with value: 1.2153575420379639.
/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]


Iter: 0, train loss: 5.6323, avg nll: 1.6496, avg kl: 3.6981, mse: 0.992594, mae: 0.790382
Iter: 100, train loss: 5.6472, avg nll: 1.5997, avg kl: 3.4324, mse: 1.009023, mae: 0.818798


[I 2022-06-23 08:37:08,729] Trial 4 finished with value: 1.4689979553222656 and parameters: {'embed_time': 64, 'enc_num_heads': 16, 'latent_dim': 64, 'lr': 4.458003216865767e-06, 'width': 128, 'num_samples': 5, 'optimizer': 'Adam', 'step_size': 400, 'gamma': 0.1482749470917385, 'beta': 0.003311582143435743}. Best is trial 0 with value: 1.2153575420379639.
/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]
[W 2022-06-23 08:37:08,944] Trial 5 failed because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 2.59 GiB (GPU 0; 15.90 GiB total capacity; 11.73 GiB already allocated; 1.96 GiB free; 13.01 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packa

RuntimeError: ignored

In [ ]:
# [I 2022-06-17 16:55:13,210] Trial 3 finished with value: 0.09719855338335037 and parameters: {'dropout': 0.02418752335062102, 'elbo_weight': 1.1061943368326204, 'embed_time': 16, 'lr': 4.212141191372822e-05, 'mse_weight': 5.108416562407145, 'rec_hidden': 128, 'num_samples': 6, 'x_by_range': True, 'optimizer': 'Adam', 'sample_tp': 0.1979453313080004}. Best is trial 3 with value: 0.09719855338335037.

optuna.visualization.plot_param_importances(study)

In [ ]:

optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study, params=["optimizer"])